In [1]:
using Pkg
using Plots
using ViscousStreaming
using NBInclude
using HDF5
using JLD2
using Distributions
using PyCall
using Distances

In [2]:
using PyCall

py"""
import random
import numpy as np
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
def set_seed(seed):
    random.seed(seed)  # Set random seed for Python's random module
    np.random.seed(seed)  # Set random seed for NumPy's random module
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def parameters():
    params = dict()
    params['seed'] = 0
    set_seed(params['seed'])
    params['episodes'] = 20000
    params['episode_length'] = 40
    params['success_rate_average_run'] = 20000
    params['success_rate_acceptable_error_ratio'] = 0.02

    params['learning_rate'] = 0.001
    params['gamma'] = 0.95
    params['memory_size'] = 500000
    params['batch_size'] = 16
    params['epsilon'] = 0.1
    params['target_model_update_iterations'] = round(params['episode_length'] / 2)

    params['first_layer_size'] = 128
    params['second_layer_size'] = 128

    params['state_size'] = 2
    params['number_of_actions'] = 16
    return params
"""

In [3]:
strdVuxy = load("strdVuxy_square.jld2")["data"];
strdVvxy = load("strdVvxy_square.jld2")["data"];

Re = 40
ϵ = 0.1
Ω = 1.0 # frequency (keep this equal to 1)
Tp = 2π/Ω # one period of oscillation
p = StreamingParams(ϵ,Re)
#s = StreamingAnalytical(p)

τ = 0.1 # Stokes number, should be small
β = 0.95 # Density parameter. Less than 1 means heavier than fluid.
p_inert = InertialParameters(tau=τ,beta=β,epsilon=ϵ,Re=Re)
Ω = 1.0
Tp = 2π/Ω
Tmax = 50*Tp

Δx = 0.02
xlim = (-2.8,2.8)
ylim = (-2.8,2.8)
n = 75
body = Circle(0.2,n)

Circular body with 72 points and radius 0.2
   Current position: (0.0,0.0)
   Current angle (rad): 0.0


In [4]:
bl = BodyList()
bL1 = deepcopy(body)
bL2 = deepcopy(body)
bR1 = deepcopy(body)
bR2 = deepcopy(body)
    
# left 1 cylinder
cent = (-2.0,2.0)
α = 0.0
TL = RigidTransform(cent,α)
TL(bL1) # transform the body to the current configuration

# left 2 cylinder
cent = (-2.0,-2.0)
α = 0.0
TL = RigidTransform(cent,α)
TL(bL2) # transform the body to the current configuration

# right 1 cylinder
cent = (2.0,2.0)
α = 0.0
TR = RigidTransform(cent,α)
TR(bR1) # transform the body to the current configuration

# right 2 cylinder
cent = (2.0,-2.0)
α = 0.0
TR = RigidTransform(cent,α)
TR(bR2) # transform the body to the current configuration

push!(bl,bL1);
push!(bl,bL2);
push!(bl,bR1);
push!(bl,bR2);

In [5]:
function mean_motion(dR,R,p,t,v̄Luxy,v̄Lvxy)
    dR[1] = v̄Luxy(R[1],R[2])
    dR[2] = v̄Lvxy(R[1],R[2])
   return dR 
end
   
function reset1()
    px = rand(Uniform(-1.8,1.8))
    py = rand(Uniform(-1.8,1.8))
    return (px,py)
end

function motion(curstate, ampvel_ind)
    newposx = 0
    newposy = 0
    done = false
    
    if ampvel_ind == 16
        newposx, newposy = curstate
    else
        v̄Lfcn(dR,R,p,t) = mean_motion(dR,R,p,t,strdVuxy[ampvel_ind],strdVvxy[ampvel_ind])
        solL = compute_trajectory(v̄Lfcn,curstate,Tmax,10Tp,bl=bl,ϵ=p.ϵ);
        newposx = last(solL[1,:])
        newposy = last(solL[2,:])
    end
    
    if newposx < -1.8  || newposx > 1.8  || newposy < -1.8  || newposy > 1.8 
        reward = -50
        newposx,newposy = curstate
    elseif newposx < 0.1  && newposx > -0.1  && newposy < 0.1  && newposy > -0.1
        reward = 2 - (abs(newposx) + abs(newposy)) 
        if newposx < 0.02  && newposx > -0.02  && newposy < 0.02  && newposy > -0.02 
            reward = 1000
            done = true
        end
    else
        reward = -1
    end 
    return (newposx,newposy), reward, done
end

motion (generic function with 1 method)